## House

In [1]:
import pandas as pd

In [2]:
df=pd.read_excel('https://labfile.oss.aliyuncs.com/courses/3023/Second_hand_house.xls')
df.head()

,省份,城市,区县,类型,小区名称,建筑面积,占地面积,绿化率,容积率,经度,纬度,价格,建成年
0,安徽,长丰,水湖镇,住宅,香溢花城,NaN,NaN,0.5100,1.90,117.170135,32.491436,NaN,2017
1,安徽,长丰,双墩镇,住宅,铭威·翰林苑,NaN,NaN,0.4366,1.89,117.170250,32.479336,NaN,2015
2,安徽,长丰,双墩镇,住宅,军港新村南区,NaN,NaN,0.3300,0.76,117.270996,32.013783,NaN,2019
3,新疆,阿克苏,阿克苏市,住宅,左岸明珠.香邸,NaN,NaN,NaN,NaN,80.274124,41.184479,NaN,2016
4,新疆,阿克苏,阿克苏市,住宅,左岸明珠·香邸,91659.0,106672.0,0.4000,3.60,80.274811,41.183842,NaN,2019


### Dimension

In [3]:
data=df
data=data.groupby(['省份'])['建筑面积'].sum().sort_values(ascending=False)
data_pair=[[prov,area] for prov,area in zip(data.index,data)]
data_pair[:5]

[['江苏', 12871059.0],
 ['山东', 6985815.0],
 ['河南', 6877657.0],
 ['广东', 6531081.37],
 ['新疆', 6065995.0]]

In [4]:
from pyecharts import options as opts
from pyecharts.charts import Map
import matplotlib.colors as cs

In [5]:
m=Map()
m.add(series_name='',
      data_pair=data_pair,
      maptype='china',
      label_opts=opts.LabelOpts(is_show=False),
      is_map_symbol_show=False)

m.set_global_opts(title_opts=opts.TitleOpts(title='Dimension Distribution',
                                            subtitle='m^2',
                                            pos_left='center',
                                            pos_top='top',
                                            title_textstyle_opts=opts.TextStyleOpts(font_size=25)),
                  visualmap_opts=opts.VisualMapOpts(is_calculable=True,
                                                    pos_left='10',
                                                    min_=100000,
                                                    max_=10000000,
                                                    range_text=['High','Low'],
                                                    range_color=[cs.TABLEAU_COLORS['tab:blue'],
                                                                 cs.TABLEAU_COLORS['tab:orange'],
                                                                 cs.TABLEAU_COLORS['tab:red']],
                                                    textstyle_opts=opts.TextStyleOpts(color='black')))
m.render_notebook()

### Bar

In [6]:
from pyecharts.charts import Bar

In [7]:
bar=Bar()
bar.add_xaxis(list(data.index))
bar.add_yaxis('Total',list(data),label_opts=opts.LabelOpts(is_show=False))

bar.set_global_opts(title_opts=opts.TitleOpts(title=' ',subtitle=' '),
                    yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(is_show=True)),
                    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=45)),
                    legend_opts=opts.LegendOpts(is_show=False),
                    visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      min_=100000,
                                                      max_=10000000,
                                                      range_text=['High','Low'],
                                                      range_color=[cs.TABLEAU_COLORS['tab:blue'],
                                                                   cs.TABLEAU_COLORS['tab:orange'],
                                                                   cs.TABLEAU_COLORS['tab:red']]))
bar.render_notebook()

### Grid

In [8]:
from pyecharts.charts import Grid

In [9]:
grid=Grid(init_opts=opts.InitOpts(width='800px',height='700px'))
grid.add(bar,grid_opts=opts.GridOpts(pos_top='60%'))
grid.add(m,grid_opts=opts.GridOpts(pos_bottom='50%'))
grid.render_notebook()

## 江苏

In [10]:
data=df
data=data.loc[data['省份']=='江苏']
data=data.groupby(['城市'])['建筑面积'].sum().sort_values(ascending=False)
data_pair=[[city+'市',area] for city,area in zip(data.index,data)]

In [12]:
m=Map()
m.add(series_name='',
      data_pair=data_pair,
      maptype='江苏',
      label_opts=opts.LabelOpts(is_show=False),
      is_map_symbol_show=False)

m.set_global_opts(title_opts=opts.TitleOpts(title='Dimension Distribution in Jiangsu',
                                            subtitle='m^2',
                                            pos_left='center',
                                            pos_top='top',
                                            title_textstyle_opts=opts.TextStyleOpts(font_size=25)),
                  visualmap_opts=opts.VisualMapOpts(is_calculable=True,
                                                    pos_left='10',
                                                    min_=1000,
                                                    max_=1550000,
                                                    range_text=['High','Low'],
                                                    range_color=[cs.TABLEAU_COLORS['tab:blue'],
                                                                 cs.TABLEAU_COLORS['tab:orange'],
                                                                 cs.TABLEAU_COLORS['tab:red']],
                                                    textstyle_opts=opts.TextStyleOpts(color='black')))

m.render_notebook()

## Price

In [13]:
from pyecharts.charts import Geo
from pyecharts.globals import ChartType

In [23]:
data=df[['小区名称','经度','纬度','价格']].copy()
data.dropna(how='any',inplace=True)

geo=Geo()
geo.add_schema(maptype='china',is_roam=True)

for name,longitude,latitude in zip(data['小区名称'],data['经度'],data['纬度']):
    geo.add_coordinate(name=name,latitude=latitude,longitude=longitude)
    
data_pair=[[name,price] for name,price in zip(data['小区名称'],data['价格'])]

geo.add('Price $Y/m^2',
        data_pair,
        type_=ChartType.HEATMAP,
        point_size=12)
geo.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
geo.set_global_opts(visualmap_opts=opts.VisualMapOpts(is_calculable=False,
                                                      dimension=0,
                                                      pos_left='10',
                                                      min_=1000,
                                                      max_=70000,
                                                      pos_top='center',
                                                      range_text=['high','low'],
                                                      range_color=['green','yellow','red']),
                    title_opts=opts.TitleOpts(title='Price Distribution'))

geo.render_notebook()

### Virescence

In [25]:
data=df[['小区名称','经度','纬度','绿化率']].copy()
data.dropna(how='any',inplace=True)

geo=Geo()
geo.add_schema(maptype='china',is_roam=True)

for name,longitude,latitude in zip(data['小区名称'],data['经度'],data['纬度']):
    geo.add_coordinate(name=name,latitude=latitude,longitude=longitude)

data_pair=[[name,green] for name,green in zip(data['小区名称'],data['绿化率'])]
geo.add('Virescence%',
        data_pair,
        type_=ChartType.HEATMAP,
        point_size=5)
geo.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
geo.set_global_opts(visualmap_opts=opts.VisualMapOpts(is_calculable=True,
                                                      dimension=0,
                                                      pos_left='10',
                                                      is_piecewise=True,
                                                      min_=0.1,
                                                      max_=0.9,
                                                      pos_top='center',
                                                      range_text=['High','Low'],
                                                      range_color=['red','yellow','green'],
                                                      textstyle_opts=opts.TextStyleOpts(color='black')),
                    title_opts=opts.TitleOpts(title='Virescence %'))
geo.render_notebook()

### Classify

In [26]:
import numpy as np

In [27]:
data=df[['小区名称','类型','经度','纬度']].copy()
data.dropna(inplace=True)

In [28]:
geo=Geo()
geo.add_schema(maptype='china',is_roam=True)

for name,longitude,latitude in zip(data['小区名称'],data['经度'],data['纬度']):
    geo.add_coordinate(name=name,latitude=latitude,longitude=longitude)

colors=[cs.TABLEAU_COLORS['tab:blue'],
        cs.TABLEAU_COLORS['tab:orange'],
        cs.TABLEAU_COLORS['tab:red'],
        cs.TABLEAU_COLORS['tab:green'],]

for i,building_type in enumerate(np.unique(data['类型'])):
    type_data=data.loc[data['类型']==building_type]
    
    geo.add(building_type,
            [[name,1] for name in type_data['小区名称']],
            type_=ChartType.SCATTER,
            symbol_size=6,
            color=colors[i])

geo.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
geo.set_global_opts(title_opts=opts.TitleOpts(title='Classifications'))
geo.render_notebook()

### Code Copy active line plots

In [29]:
from pyecharts.globals import SymbolType

lines = ['上海', '昆明', '乌鲁木齐', '郑州', ]

lines = [[start, stop] for start, stop in zip(
    np.array(lines), np.append(lines[1:], lines[0]))]

geo=Geo()
geo.add_schema(maptype='china',is_roam=True)
geo.add('',
        lines,
        type_=ChartType.LINES,
        effect_opts=opts.EffectOpts(symbol=SymbolType.ARROW,
                                    symbol_size=6,
                                    color=cs.TABLEAU_COLORS['tab:blue']),
        linestyle_opts=opts.LineStyleOpts(curve=.2,
                                          color=cs.TABLEAU_COLORS['tab:red']))
geo.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
geo.set_global_opts(title_opts=opts.TitleOpts(title='Line'))
geo.render_notebook()

### Price Change by Yr

In [30]:
data=df.copy()
data=data[['价格','省份','建成年']]
data.dropna(inplace=True)
data['建成年']=data['建成年'].apply(lambda x:int(x))
data.head()

,价格,省份,建成年
6,7912.0,新疆,2015
7,9675.0,西藏,2017
11,6974.0,陕西,2016
16,10680.0,安徽,2016
43,3166.0,陕西,2016


In [32]:
from pyecharts.charts import Timeline

In [34]:
timeline=Timeline(init_opts=opts.InitOpts(width='800px',height='700px',theme='dark'))
timeline.add_schema(play_interval=900)

for year in sorted(np.unique(data['建成年'])):
    year_data=data.loc[data['建成年']==year]
    year_data=year_data.groupby(['省份'])['价格'].mean().sort_values(ascending=False)
    data_pair=[[prov,price] for prov,price in zip(year_data.index,year_data)]
    
    m=Map()
    m.add(series_name='',
      data_pair=data_pair,
      label_opts=opts.LabelOpts(is_show=False),
      is_map_symbol_show=False)
    m.set_global_opts(title_opts=opts.TitleOpts(title='Price by Year',
                                            subtitle='$Y/m^2',
                                            pos_left='center',
                                            pos_top='top',
                                            title_textstyle_opts=opts.TextStyleOpts(font_size=25)),
                  visualmap_opts=opts.VisualMapOpts(is_calculable=True,
                                                    dimension=0,
                                                    pos_left='10',
                                                    min_=year_data.min(),
                                                    max_=year_data.max(),
                                                    pos_top='center',
                                                    range_text=['High','Low'],
                                                    range_color=[cs.TABLEAU_COLORS['tab:blue'],
                                                                 cs.TABLEAU_COLORS['tab:orange'],
                                                                 cs.TABLEAU_COLORS['tab:red']],
                                                    textstyle_opts=opts.TextStyleOpts(color='#ddd')))

    bar=Bar()
    bar.add_xaxis(list(year_data.index))
    bar.add_yaxis('Overall',list(year_data),label_opts=opts.LabelOpts(is_show=False))
    bar.set_global_opts(title_opts=opts.TitleOpts(title='',
                                              subtitle=''),
                    yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(is_show=True)),
                    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=45)),
                    legend_opts=opts.LegendOpts(is_show=False),
                    visualmap_opts=opts.VisualMapOpts(
                        is_show=False,
                        min_=year_data.min(),
                        max_=year_data.max(),
                        range_text=["High", "Low"],
                        range_color=[cs.TABLEAU_COLORS['tab:blue'],
                                     cs.TABLEAU_COLORS['tab:orange'],
                                     cs.TABLEAU_COLORS['tab:red']]))

    grid=Grid()
    grid.add(bar,grid_opts=opts.GridOpts(pos_top='65%',pos_bottom='10%'))
    grid.add(m,grid_opts=opts.GridOpts(pos_bottom='50%'))
    timeline.add(grid,f'Year {year}')
timeline.render_notebook()